In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
hc = 12.39841875 #keV*Angström
d_si = 3.1358

#Implementirung der Funktionen für die Auswertung

## Die Peak sollen mit der Gaussverteilung angefitten werde, diese wird hier implementier, wie ein wrappen für automatische fits

#Gauss
<h1>$f(x) = \frac{a}{\sqrt{2 \cdot pi \cdot \sigma^2}} \cdot e^{- \frac{{(x- \sigma )}^2}{2 \cdot \sigma^2}} $ </h1>

In [3]:
def gauss(x,a,mu,sigma):
    return (a/(np.sqrt(2*3.14*sigma**2))*np.exp(-((x-mu)**2)/(2.*sigma**2)))

#${\chi_{red}}^2$ / Goodness of a fit
<h1> ${\chi_{red}}^2 = \frac{1}{\nu}\sum\frac{{(O-E)}^2}{\sigma^2}$ 

In [4]:
def red_chi_gauss(data, model, deg):
    chisq = np.sum(np.square(data-model)/np.power(data,0.5))
    nu = data.size-1-deg    # -4: -1 for the equation and -3 from the degrees of freedom
    if nu is 0:
        return "err: nu = 0"
    return chisq/nu

In [5]:
def fit_gauss(x,data,guess=[1,1,1]):
    '''
        returns the parameter from the fit, the standard deviation and the reduced chisquare
    '''
    para, cov = opt.curve_fit(gauss,x,np.sqrt(data),guess,maxfev=10000)
    red_chi = red_chi_gauss(data,gauss(x,para[0],para[1],para[2]),3)
    return para, np.sqrt(np.diag(cov)), red_chi

# Bragg-reflexion
$n \cdot \lambda = 2 \cdot d \cdot sin(\theta)$

# Energie
$E = \frac{h \cdot c}{2 \cdot d \cdot sin(\theta)}$

In [6]:
def energie(phi,delta_phi):
    theta = phi/2
    e = hc/(2*d_si*np.sin(theta*(3.14/180)))
    delta_e = (e*delta_phi/2)/np.tan(theta*(3.14/180))
    return e, delta_e

In [7]:
def intensität(a,delta_a,sigma,delta_sigma):
    i = a/(np.sqrt(2*3.14*sigma**2))
    delta_i = np.sqrt((delta_a/np.sqrt(2*3.14*sigma**2))**2 + ((a*delta_sigma)/(np.sqrt(2*3.14*sigma**4)))**2)
    return i, delta_i

In [8]:
def d(e,delta_e,phi,delta_phi,n):
    d = (n*hc)/(2*e*np.sin((phi/2)*(3.14/180)))
    delta_d = d*np.sqrt(((delta_phi/2)/np.tan((phi/2)*(3.14/180)))**2+(delta_e/e)**2)

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = ??, A$_1$ = ??
+ U$_2$ = ??, A$_2$ = ??
+ U$_3$ = ??, A$_3$ = ??

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdefraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristall größe Ermitteln

# Ersterwert: Spannung
# Zweiterwert: Anodenstrom


# Vorbereitung für den Voigt-Fit

In [9]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')

In [155]:
data = np.loadtxt("Si111_40_30.xy").T
plt.plot(data[0],data[1])
plt.show()

In [154]:
params_1 = voigt_1.guess(data[1],x=data[0])
params_2 = voigt_2.guess(data[1],x=data[0])
params_3 = voigt_3.guess(data[1],x=data[0])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [72]:
fit_1 = voigt_1.fit(data[1],params_1,x=data[0])
fit_2 = voigt_2.fit(data[1],params_2,x=data[0])
fit_3 = voigt_3.fit(data[1],params_3,x=data[0])

In [219]:
peak_1_range = np.arange(np.where( 25.5 < data[0])[0][0],np.where( data[0] > 25.8)[0][0])
peak_2_range = np.arange(np.where( 28.39 < data[0])[0][0],np.where( data[0] > 28.48)[0][0])
peak_3_range = np.arange(np.where( 28.47 < data[0])[0][0],np.where( data[0] > 28.54)[0][0])

In [227]:
params_1 = voigt_1.guess(data[1][peak_1_range],x=data[0][peak_1_range])
params_2 = voigt_2.guess(data[1][peak_2_range],x=data[0][peak_2_range])
params_3 = voigt_3.guess(data[1][peak_3_range],x=data[0][peak_3_range])

params_1['voigt_1gamma'].set(1,vary=True,expr='')
params_2['voigt_2gamma'].set(1,vary=True,expr='')
params_3['voigt_3gamma'].set(1,vary=True,expr='')

In [230]:
fit_1 = voigt_1.fit(data[1][peak_1_range],params_1,x=data[0][peak_1_range],weights=1/np.sqrt(data[1][peak_1_range]*100))
fit_2 = voigt_2.fit(data[1][peak_2_range],params_2,x=data[0][peak_2_range],weights=1/np.sqrt(data[1][peak_2_range]*100))
fit_3 = voigt_3.fit(data[1][peak_3_range],params_3,x=data[0][peak_3_range],weights=1/np.sqrt(data[1][peak_3_range]*100))

In [231]:
fit_1.plot()
fit_2.plot()
fit_3.plot()
plt.show()

In [232]:
print(fit_1.fit_report())
print(fit_2.fit_report())
print(fit_3.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 170
    # data points      = 36
    # variables        = 4
    chi-square         = 39.124
    reduced chi-square = 1.223
[[Variables]]
    voigt_1fwhm:        0.02609075 +/- 0.005542 (21.24%)  == '3.6013100*voigt_1sigma'
    voigt_1center:      25.6475699 +/- 0.000527 (0.00%) (init= 25.64677)
    voigt_1amplitude:   1913.71343 +/- 46.15069 (2.41%) (init= 1315.524)
    voigt_1sigma:       0.00724479 +/- 0.001538 (21.24%) (init= 0.008146744)
    voigt_1gamma:       0.01352327 +/- 0.000889 (6.57%) (init= 1)
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.793 
    C(voigt_1amplitude, voigt_1gamma)  =  0.162 
    C(voigt_1amplitude, voigt_1sigma)  = -0.128 
[[Model]]
    Model(voigt, prefix='voigt_2')
[[Fit Statistics]]
    # function evals   = 288
    # data points      = 11
    # variables        = 4
    chi-square         = 20.668
    reduced chi-squar

In [238]:
plt.errorbar(data[0],data[1],np.sqrt(data[1]))
plt.plot(data[0][peak_1_range],fit_1.best_fit)
plt.plot(data[0][peak_2_range],fit_2.best_fit)
plt.plot(data[0][peak_3_range],fit_3.best_fit)
plt.show()